# Execute SQL with Python in SAS Viya Workbench

Import the necessary packages.

In [ ]:
import requests
import pandas as pd
from pandasql import sqldf

## 1.  Download the data to the SAS Viya Workbench environment

In [ ]:
## File path and name
file_path = r"/workspaces/myfolder/ExecutingSQL/home_equity_py.csv"

## Specify the URL of the CSV file
url = r"https://support.sas.com/documentation/onlinedoc/viya/exampledatasets/home_equity.csv"

## Download the and save CSV file to Workbench
response = requests.get(url)
with open(file_path, 'wb') as f:
    f.write(response.content)
    print(f'File downloaded:{file_path}')


## 2. Create the DataFrame

Read the **home_equity_py.csv** file from the **ExecutingSQL** folder.

In [ ]:
he_df = pd.read_csv(file_path)

Preview the DataFrame.

In [ ]:
he_df.head()

## 3. Execute SQL using the pandasql package

Preview the data using SQL using the LIMIT clause.

In [ ]:
sqldf('''
    SELECT *
    FROM he_df
    LIMIT 10
''')

Store the query in a variable as a string for readability.

In [ ]:
preview_data = '''
SELECT *
FROM he_df
LIMIT 10
'''
sqldf(preview_data)

Use the GROUP BY clause to aggregate your data.

In [ ]:
query = '''
SELECT BAD, count(*) as TOTAL
FROM he_df
GROUP BY BAD;
'''

sqldf(query)

Count the number of missing values in the **BAD**, **MORTDUE**, and **VALUE** columns.

In [ ]:
query = '''
SELECT
    sum(CASE WHEN BAD IS NULL THEN 1 ELSE 0 END) as MISSING_BAD,
    sum(CASE WHEN VALUE IS NULL THEN 1 ELSE 0 END) as MISSING_VALUE,
    sum(CASE WHEN MORTDUE IS NULL THEN 1 ELSE 0 END) as MISSING_MORTDUE
FROM he_df;
'''

sqldf(query)

Use Python functions and f-strings to create dynamic code. Create a function that calculates the total number of NULL values in a specified column.

In [ ]:
def num_missing(df, colName):

    df_copy = df

    query = f'''
    SELECT
        sum(CASE WHEN {colName} IS NULL THEN 1 ELSE 0 END) as MISSING_{colName}
    FROM df_copy;
    '''
    return sqldf(query)

In [ ]:
num_missing(he_df,'MORTDUE')

Use Python f-strings and list comprehensions to create a dynamic function to count the total number of NULL values in every column of a DataFrame.

In [ ]:
def count_all_missing(df):

    df_copy = df

    ## List of column names in the DataFrame
    colNames = df_copy.columns.to_list()

    ## Create the CASE expression SELECT statement for each column
    cols_case_statement_list = [f'SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) as MISSING_{col}' for col in colNames]
    
    ## Create a string using each element of the list, separated by a comma
    delimiter = ", "
    select_stm = delimiter.join(cols_case_statement_list)
    
    ## Add the SELECT statement to the query
    query = f'SELECT {select_stm} FROM df_copy'

    return sqldf(query)

In [ ]:
count_all_missing(he_df)